# Fine-tune Prophetnet on Xsum smallset

##Libraries and environment preparation

In [1]:
#Install essential packages
%%capture
!pip install datasets transformers rouge-score nltk wandb
!apt install git-lfs

In [2]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

Wed Mar  9 15:24:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

In [3]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.17.0


In [4]:
from transformers import AutoTokenizer    
# Huggingface Automodel class
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "microsoft/prophetnet-large-uncased"

In [5]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

##Load the dataset

In [6]:
# import dataset
raw_datasets = datasets.load_dataset("xsum")

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

##Preprocess the data

In [8]:
# Import tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/141 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [9]:
# tokenlize inputs into map

max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_train = raw_datasets["train"].select(range(120000)).map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["document", "summary", "id"])

  0%|          | 0/120 [00:00<?, ?ba/s]

In [11]:
tokenized_val = raw_datasets["validation"].select(range(2000)).map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["document", "summary", "id"])

  0%|          | 0/2 [00:00<?, ?ba/s]

In [12]:
tokenized_train

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 120000
})

##Fine-tuning the model

In [13]:
# Import tokenizer from model checkpoint and print detail
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, use_cache=False)

# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

In [14]:
wandb.init(project="prophetnet")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


In [15]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 4
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    load_best_model_at_end="eval_loss",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=1000, 
    save_steps=4000,
    eval_steps=4000,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

In [16]:
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [17]:
# Pass into the trainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [18]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ProphetNetForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `ProphetNetForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 120000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 30000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
4000,2.988400,2.786937,35.623900,13.561600,28.409800,28.463100,24.482000
8000,2.900800,2.689570,36.779100,14.213700,29.243200,29.279100,25.847000
12000,2.810700,2.633083,37.184000,14.563200,29.479100,29.497200,25.407000
16000,2.711500,2.547688,38.582400,15.760300,30.725500,30.755500,26.353000
20000,2.662700,2.468134,39.801300,16.601400,31.944000,31.959900,25.737000
24000,2.596400,2.395490,40.083400,17.094600,32.413000,32.423900,25.814000
28000,2.503500,2.348928,40.978400,18.110400,33.369700,33.380500,25.672500


The following columns in the evaluation set  don't have a corresponding argument in `ProphetNetForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `ProphetNetForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 4
Saving model checkpoint to prophetnet-large-uncased-finetuned-xsum/checkpoint-4000
Configuration saved in prophetnet-large-uncased-finetuned-xsum/checkpoint-4000/config.json
Model weights saved in prophetnet-large-uncased-finetuned-xsum/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in prophetnet-large-uncased-finetuned-xsum/checkpoint-4000/tokenizer_config.json
Special tokens file saved in prophetnet-large-uncased-finetuned-xsum/checkpoint-4000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `ProphetNetForConditionalGeneration.forward` and have been ignored: 

TrainOutput(global_step=30000, training_loss=2.75568388671875, metrics={'train_runtime': 42830.9916, 'train_samples_per_second': 2.802, 'train_steps_per_second': 0.7, 'total_flos': 1.270789152326615e+17, 'train_loss': 2.75568388671875, 'epoch': 1.0})

In [19]:
wandb.finish()

eval/gen_len,▁▆▄█▆▆▅
eval/loss,█▆▆▄▃▂▁
eval/rouge1,▁▃▃▅▆▇█
eval/rouge2,▁▂▃▄▆▆█
eval/rougeL,▁▂▃▄▆▇█
eval/rougeLsum,▁▂▂▄▆▇█
eval/runtime,▁▅▄█▄▃▃
eval/samples_per_second,█▄▅▁▅▆▅
eval/steps_per_second,█▅▆▁▆▆▆
train/epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████


In [21]:
!ls -lh prophetnet-large-uncased-finetuned-xsum

total 12K
drwxr-xr-x 2 root root 4.0K Mar  9 23:58 checkpoint-20000
drwxr-xr-x 2 root root 4.0K Mar 10 01:36 checkpoint-24000
drwxr-xr-x 2 root root 4.0K Mar 10 03:13 checkpoint-28000


In [22]:
!zip -r prophetnet-large-uncased-finetuned-xsum.zip /content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/

  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/ (stored 0%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/scheduler.pt (deflated 49%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/tokenizer_config.json (deflated 38%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/trainer_state.json (deflated 81%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/special_tokens_map.json (deflated 37%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/prophetnet.tokenizer (deflated 53%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/training_args.bin (deflated 48%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/optimizer.pt (deflated 9%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-28000/scaler.pt (deflated 55%)
  adding: content/prophetnet-large-uncased-finetuned-

In [23]:
!cp prophetnet-large-uncased-finetuned-xsum.zip '/content/drive/My Drive/weights/'